## River
Библиотека, которая позволяет обучать модели с помощью потоковых данных. Ниже приведен пример с логистической регрессией, которая определяет ялвяется ли сайт фишинговым.

In [2]:
!pip install river

  Using cached river-0.8.0-cp38-cp38-win_amd64.whl (1.4 MB)


In [1]:
"""
    dataset - это не загруженный набор данных, а поток.
    for x, y in datasets: - чтение потока данных, или, проще говоря, чтение всего файла.
    model.learn_one(x, y) - говорит о том, что модель можно в будущем дообучать.
    metric.update(y, y_pred) - метрику также можно уточнять с течением времени.
    model.predict_proba_one(x) - предсказывает вероятность классификации классов.
"""

from river import datasets
from river import linear_model
from river import metrics


dataset = datasets.Phishing()
model = linear_model.LogisticRegression()
metric = metrics.ROCAUC()

for x, y in dataset:
    y_pred = model.predict_proba_one(x)
    model.learn_one(x, y)
    metric.update(y, y_pred)

print(metric)

ROCAUC: 0.893565


## Чтение данных

В river признаки сэмпла хранятся внутри словаря, который в Python называется dict и является собственной структурой данных. Другими словами, мы не используем никаких сложных структур данных, таких как numpy.ndarray или pandas.DataFrame. 

Какие у этого преимущества? Одно из преимуществ заключается в том, что dict устраняет накладные расходы, связанные с использованием вышеупомянутых структур данных. Это дает нам скорость в обучении модели.

Следует отметить, что уже готовый датасет datasets.Phishing() мы можем прочитать самостоятельно, а следовательно и любой другой csv и тем самым настроить потоковое чтение.

In [2]:
dataset = datasets.Bikes()
for x, y in dataset:
    pass
x

{'moment': datetime.datetime(2016, 10, 5, 9, 57, 18),
 'station': 'pomme',
 'clouds': 88,
 'description': 'overcast clouds',
 'humidity': 84,
 'pressure': 1017.34,
 'temperature': 17.45,
 'wind': 1.95}

#### С помощью river.stream.iter_csv(...) можно настроить потоковое чтение.

In [3]:
from river import stream

ds = stream.iter_csv(dataset.path)
x, y = next(ds)
x

{'moment': '2016-04-01 00:00:07',
 'bikes': '1',
 'station': 'metro-canal-du-midi',
 'clouds': '75',
 'description': 'light rain',
 'humidity': '81',
 'pressure': '1017.0',
 'temperature': '6.54',
 'wind': '9.3'}

#### Однако все поля имеют строковый тип, поэтому нужно добавить дополнительные настройки.

In [4]:
ds = stream.iter_csv(
    dataset.path,
    converters={
        'bikes': int,
        'clouds': int,
        'humidity': int,
        'pressure': float,
        'temperature': float,
        'wind': float
    },
    parse_dates={'moment': '%Y-%m-%d %H:%M:%S'},
    target='bikes')

x, y = next(ds)
x

{'moment': datetime.datetime(2016, 4, 1, 0, 0, 7),
 'station': 'metro-canal-du-midi',
 'clouds': 75,
 'description': 'light rain',
 'humidity': 81,
 'pressure': 1017.0,
 'temperature': 6.54,
 'wind': 9.3}

## Pipeline
Compose.Pipeline содержит всю логику для построения и применения конвейеров. Конвейер - это, по сути, список оценщиков, которые применяются последовательно. Единственное требование - чтобы первые $n - 1$ ступень были трансформаторами. Последним шагом может быть регрессор, классификатор, кластеризатор, преобразователь и т.д. Вот пример: 

In [5]:
from river import compose
from river import preprocessing
from river import feature_extraction

model = compose.Pipeline(
    preprocessing.StandardScaler(),
    feature_extraction.PolynomialExtender(),
    linear_model.LinearRegression()
)

# можно визуализировать построенную модель
model

Pipeline (
  StandardScaler (
    with_std=True
  ),
  PolynomialExtender (
    degree=2
    interaction_only=False
    include_bias=False
    bias_name="bias"
  ),
  LinearRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.01
      )
    )
    loss=Squared ()
    l2=0.
    intercept_init=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

Также можно строить Pipeline и другим способом

In [6]:
model = (
    preprocessing.StandardScaler() |
    feature_extraction.PolynomialExtender() |
    linear_model.LinearRegression()
)

model

Pipeline (
  StandardScaler (
    with_std=True
  ),
  PolynomialExtender (
    degree=2
    interaction_only=False
    include_bias=False
    bias_name="bias"
  ),
  LinearRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.01
      )
    )
    loss=Squared ()
    l2=0.
    intercept_init=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

## MLPRegressor

In [7]:
from river import datasets
from river import evaluate
from river import neural_net as nn
from river import optim
from river import preprocessing as pp
from river import metrics

model = (
    pp.StandardScaler() |
    nn.MLPRegressor(
        hidden_dims=(5,),
        activations=(
            nn.activations.ReLU,
            nn.activations.ReLU,
            nn.activations.Identity
        ),
        optimizer=optim.SGD(1e-3),
        seed=42
    )
)

dataset = datasets.TrumpApproval()

metric = metrics.MAE()

evaluate.progressive_val_score(dataset, model, metric)

MAE: 1.589827